In [1]:
## import system installed packages.
import os                # for filesystem access
import sys               # for system sensors on flowing tensors.
import tensorflow as tf  # lets the tensors flow.
import numpy as np       # most tf objects are built from numpy.
import pandas as pd      # data holder manager.
from pandas.io.json import json_normalize  # json to pandas converter.
from datetime import datetime, timedelta   # need unix 2 real and visa-versa.
import time                 # i never know which time is better for what... but there is no time for that...
import json                 # data from web comes as json.
import urllib               # for web calls.
from urllib import request  # for secure web connections.

In [2]:
## we are gonna try to NOT use anything that isnt pypy installable or on
## this page... This page is going to get long i think. from this point,
## setting up ALL the global variables, like fs_paths, save points, how
## much work to do, web urls if necessary, global stuff.

In [3]:
class Options(object):
    """An object that contains all global variables that might be needed,
       in various locations.
       
       A good way to put the human readable controls upfront where the
       driver can find them.
    """

    def __init__(self):
        """All public options go here."""
        self.model_save_path = 'save_path'  # NOT SET
        self.data_save_path = 'data_path'   # NOT SET
        self.data_filename = 'poloniex_historical_data_all_{}.npz'
        
        #### poloniex api variables #####
        self._coin = 'BTC_ETH'
        self._candle_period = 300
        # about 2 weeks ... now - a million seconds
        self._start_time = int(time.time()) - 1000000
        # 9999999999 means most current by poloniex standard api
        # see how im mixing time libs cause i dont know whats better?
        self._end_time = int(time.time())

    def __str__(self):
        """Make sense from calling an object, not a memory position."""
        return self.model_save_path

    def __call__(self):
        """Directly print out all available options."""
        print("##########\n#\tAll Available Options:")
        for i in sorted(self.__dict__):
            print("#\t{}:  {}".format(i, self.__dict__['{}'.format(i)]))
### The options CLASS is refered to as the CONFIG(config)...
config = Options()

In [4]:
### Print out all the config usage and variables.
while False:
    print("Config itself is reference to the model save path:\n\t{}".format(
        config))
    print("Calling config will print a list of all the options for reference.")
    config()
    break;

In [5]:
### No Api credentials are necessary, poloniex would like to see,
### less than 20 api calls a minute... so sleep for 3 secs is best,
### BUT... i think you can get 20 a sec at most... but I wouldnt push it.
def quote_history(coin="BTC_ETH", period=900, start=00, end=9999999999):
    """Poloniex public chart data with candlesticks."""
    # start = int(time.time() - 1000000)
    with urllib.request.urlopen(
        'https://poloniex.com/public?command=returnChartData&currencyPair={}&start={}&end={}&period={}'.format(
        coin, start, end, period
        )
                    ) as url:
        data = json.loads(url.read().decode('UTF-8'))
    return data

def ticker():
    """Poloniex public ticker."""
    with urllib.request.urlopen(
        'https://poloniex.com/public?command=returnTicker'
                    ) as url:
        data = json.loads(url.read().decode('UTF-8'))
    return data

while False:
    # test to make sure we are getting the api call right.
    print(quote_history(config._coin, 
                        config._candle_period, 
                        config._start_time+99000,
                        config._end_time))
    print(ticker())
    break;

In [6]:
### poloniex reports everything in unix time. But plotting in real-time
### is better. So we need to convert back and forth.

In [7]:
def to_unix_time(dt):
    """Take a datetime formated string and return a unix time."""
    epoch =  datetime.utcfromtimestamp(0)
    return (dt - epoch).total_seconds() * 1000
def to_date_time(ut):
    """You guessed it! take a unix time string.
       THEN handle with care(float,int), then return just the date.
    """
    return datetime.fromtimestamp(int(float(ut))).strftime('%Y-%m-%d')

In [8]:
### GENERALLY using of a pandas dataframe accessing the Yahoo stock api
### is what is used in most tutorials. Now that our poloniex apicall is 
### real similiar to the yahoo one. we should be able to use this data in
### real-time.

In [9]:
# Collect a test dataframe from the exchange
df = json_normalize(quote_history(config._coin,
                                  config._candle_period,
                                  config._start_time,
                                  config._end_time))

In [10]:
### print out some information about the dataframe.
while True:
    """Make sure the data is where its supposed to be."""
    # print(df)
    # start with the 0th date
    start_date = to_date_time(df['date'][0])
    # end with -1th date
    end_date = to_date_time(df['date'][-1:])
    date_period = timedelta(seconds=int(float(df['date'][-1:])) - int(float(df['date'][0])))
    print("Time Period: {}\nStart Date: {}\nEnd Date: {}".format(
        date_period,
        start_date,
        end_date
    ))
    print('Columns to explore:')
    for i in df:
        print('\t- ', i)
    break;

Time Period: 11 days, 13:45:00
Start Date: 2017-07-05
End Date: 2017-07-17
Columns to explore:
	-  close
	-  date
	-  high
	-  low
	-  open
	-  quoteVolume
	-  volume
	-  weightedAverage


In [11]:
### IN plotly i needed to update all the unix times to real-times but i,
### dont know if that is the case here. so we can leave it in... and false
### for now.
while False:
    # need to update away from unixtime and into realtime
    print('Changing from unix time to datetime...\nThis will take a sec...')
    for index, key in enumerate(df['date']):
        df['date'][index] = to_date_time(key)
    print('Finished Changing dates')
    break;

Inventory of objects:
    - config
    - df
    - quote_history()
Nearly prepared for tensorflow. but we still need a saving data mechinizm.
Needs:
    - save to a pickle or npz object.
    - save to redis or mongo or mysql

In [12]:
### OKAY... BRIGHT IDEA!!! how about i just... GRAB ALL THAT DATA ONCE AND
### FOR ALL... stuff it into a numpy object... boom all_data.npz
### i need the ticker data for that. then candles 300 - 5 mins... for real...
### all time quote for each one in the ticker...

In [13]:
# single_tick = json_normalize(ticker())

In [14]:
single_tick = pd.DataFrame(ticker())

In [15]:
### print out some information about the dataframe.
while True:
    """Make sure the data is where its supposed to be."""
    # print(single_tick)
    print('rows to explore: {}'.format(len(single_tick)))
    col = 0
    for i, e in enumerate(single_tick):
        if i in range(1):
            print('{}'.format(e))
            print(single_tick[e])
            #for j in single_tick[e]:
                #print(j)
        col += 1
    print('Columns to explore: {}'.format(col))
    break;

rows to explore: 10
BTC_AMP
baseVolume           117.39029492
high24hr               0.00008425
highestBid             0.00008302
id                            160
isFrozen                        0
last                   0.00008309
low24hr                0.00007209
lowestAsk              0.00008309
percentChange          0.14040625
quoteVolume      1473617.95229485
Name: BTC_AMP, dtype: object
Columns to explore: 90


In [29]:
### find out arbritrage delta
# starting balances
start_bal_zec = .2 # zec
start_bal_usdt = 0
start_bal_btc = 0
# last loop
last_bal_zec = 0 # zec
last_bal_usdt = 0
last_bal_btc = 0
# curr loop
cur_bal_zec = 0.2 # zec
cur_bal_usdt = 0
cur_bal_btc = 0

# timers
cur_index = 0

# globals
l = 'last'
# fee is .25% in whatever you are buying
fee = .0025

# placeholders
dataset = dict()

# abrtiage delta value
ABT = 0

In [30]:
if True:
    # starting sequence... always sleep to not piss off api
    # time.sleep(2)
    
    # get the data
    single_tick = pd.DataFrame(ticker())
    for i in single_tick:
        x = single_tick[i][l]
        if 'BTC_ZEC' in i or 'USDT_BTC' in i or 'USDT_ZEC' in i:
            # print("{}\n\t{}: {}".format(i, l, x))
            dataset[i] = x
            
    ##
    print("#  | ABT for USDT holdings | #")
    # zec -> btc -> usdt : zec -> usdt -> zec
    #zec_usdt = float(dataset['USDT_ZEC']) * bal_zec
    #ex1_fee = zec_usdt * fee
    #zec_usdt -= ex1_fee
    #print("total usdt bal after zec sale: {:.8f}, fee: {:.3f}".format(zec_usdt, ex1_fee))
    
    """now zec -> btc"""
    # adjust balances
    last_bal_btc = cur_bal_btc
    cur_bal_btc = float(single_tick['BTC_ZEC']['lowestAsk']) * cur_bal_zec
    last_bal_zec = float(cur_bal_zec)
    cur_bal_zec = 0
    
    # subtract fees
    ex2_fee = cur_bal_btc * fee
    cur_bal_btc -= ex2_fee
    
    # final
    print("sale:{:.8f} ZEC for {:.8f} BTC, fee: {:.8f} BTC".format(last_bal_zec, cur_bal_btc, ex2_fee))
    
    """now btc -> USDT"""
    # adjust balances
    last_bal_usdt = cur_bal_usdt
    cur_bal_usdt = float(single_tick['USDT_BTC']['lowestAsk']) * cur_bal_btc
    last_bal_btc = cur_bal_btc
    cur_bal_btc = 0
    
    # subtract fees    
    ex3_fee = cur_bal_usdt * fee
    cur_bal_usdt -= ex3_fee
    
    # final
    print("sale:{:.8f} BTC for {:.3f} USDT, fee: {:.3f} USDT".format(last_bal_btc, cur_bal_usdt, ex3_fee))
    
    """now USDT -> ZEC"""
    # adjust balances
    cur_bal_zec = cur_bal_usdt / float(single_tick['USDT_ZEC']['lowestAsk'])
    last_bal_usdt = cur_bal_usdt
    cur_bal_usdt = 0
    
    # subtract fees
    ex4_fee = cur_bal_zec * fee
    cur_bal_zec -= ex4_fee
    
    # final
    print("sale:{:.3f} BTC for {:.8f} ZEC, fee: {:.8f} ZEC".format(last_bal_usdt, cur_bal_zec, ex4_fee))
    
    abt = 100*(cur_bal_zec / last_bal_zec)
    print("Current abt: {:.4f}%".format(abt))
    if abt > 100.5:
        print("starting abt strategy")
    else:
        print("No oppertunity for abt: ZEC -> BTC -> USDT -> ZEC")
    
    # transaction time...
    # time.sleep(5)

#  | ABT for USDT holdings | #
sale:0.20000000 ZEC for 0.01633047 BTC, fee: 0.00004093 BTC
sale:0.01633047 BTC for 33.475 USDT, fee: 0.084 USDT
sale:33.475 BTC for 0.19805192 ZEC, fee: 0.00049637 ZEC
Current abt: 99.0260%
No oppertunity for abt: ZEC -> BTC -> USDT -> ZEC


In [ ]:
if True:
    # starting sequence... always sleep to not piss off api
    # time.sleep(2)
    print("#  | ABT for USDT holdings | #")
    
    """zec -> btc"""
    # adjust balances
    last_bal_btc = cur_bal_btc
    cur_bal_btc = float(single_tick['BTC_LTC']['lowestAsk']) * cur_bal_zec
    last_bal_zec = float(cur_bal_zec)
    cur_bal_zec = 0
    
    # subtract fees
    ex2_fee = bal_btc * fee
    cur_bal_btc -= ex2_fee
    
    # final
    print("sale:{:.8f} ZEC for {:.8f} BTC, fee: {:.8f} BTC".format(last_bal_zec, cur_bal_btc, ex2_fee))
    
    """btc -> USDT"""
    # adjust balances
    last_bal_usdt = cur_bal_usdt
    cur_bal_usdt = float(single_tick['USDT_LTC']['lowestAsk']) * cur_bal_btc
    last_bal_btc = cur_bal_btc
    cur_bal_btc = 0
    
    # subtract fees    
    ex3_fee = cur_bal_usdt * fee
    cur_bal_usdt -= ex3_fee
    
    # final
    print("sale:{:.8f} BTC for {:.3f} USDT, fee: {:.3f} USDT".format(last_bal_btc, cur_bal_usdt, ex3_fee))
    
    """USDT -> zec"""
    # adjust balances
    cur_bal_zec = cur_bal_usdt / float(single_tick['USDT_LTC']['lowestAsk'])
    last_bal_usdt = cur_bal_usdt
    cur_bal_usdt = 0
    
    # subtract fees
    ex4_fee = cur_bal_zec * fee
    cur_bal_zec -= ex4_fee
    
    # final
    print("sale:{:.3f} BTC for {:.8f} ZEC, fee: {:.8f} ZEC".format(last_bal_usdt, cur_bal_zec, ex4_fee))
    
    # abt is new price / old price ... if > 1 abt is a go.
    abt = cur_bal_zec / last_bal_zec
    print("Current abt: {:.4f}%".format(abt))
    if abt > 1:
        print("starting abt strategy")
    else:
        print("No oppertunity for abt: ZEC -> BTC -> USDT -> ZEC")
    
    # transaction time...
    # time.sleep(5)

In [ ]:
if True:
    # starting sequence... always sleep to not piss off api
    # time.sleep(2)
    print("#  | ABT for USDT holdings | #")
    
    """zec -> btc"""
    # adjust balances
    last_bal_btc = cur_bal_btc
    cur_bal_btc = float(single_tick['BTC_XMR']['lowestAsk']) * cur_bal_zec
    last_bal_zec = float(cur_bal_zec)
    cur_bal_zec = 0
    
    # subtract fees
    ex2_fee = bal_btc * fee
    cur_bal_btc -= ex2_fee
    
    # final
    print("sale:{:.8f} ZEC for {:.8f} BTC, fee: {:.8f} BTC".format(last_bal_zec, cur_bal_btc, ex2_fee))
    
    """btc -> USDT"""
    # adjust balances
    last_bal_usdt = cur_bal_usdt
    cur_bal_usdt = float(single_tick['USDT_XMR']['lowestAsk']) * cur_bal_btc
    last_bal_btc = cur_bal_btc
    cur_bal_btc = 0
    
    # subtract fees    
    ex3_fee = cur_bal_usdt * fee
    cur_bal_usdt -= ex3_fee
    
    # final
    print("sale:{:.8f} BTC for {:.3f} USDT, fee: {:.3f} USDT".format(last_bal_btc, cur_bal_usdt, ex3_fee))
    
    """USDT -> zec"""
    # adjust balances
    cur_bal_zec = cur_bal_usdt / float(single_tick['USDT_XMR']['lowestAsk'])
    last_bal_usdt = cur_bal_usdt
    cur_bal_usdt = 0
    
    # subtract fees
    ex4_fee = cur_bal_zec * fee
    cur_bal_zec -= ex4_fee
    
    # final
    print("sale:{:.3f} BTC for {:.8f} ZEC, fee: {:.8f} ZEC".format(last_bal_usdt, cur_bal_zec, ex4_fee))
    
    # abt is new price / old price ... if > 1 abt is a go.
    abt = cur_bal_zec / last_bal_zec
    print("Current abt: {:.4f}%".format(abt))
    if abt > 1:
        print("starting abt strategy")
    else:
        print("No oppertunity for abt: ZEC -> BTC -> USDT -> ZEC")
    
    # transaction time...
    # time.sleep(5)

In [ ]:
#### get into the filesystem and get our data if we need it.
if False:
    if not os.path.isdir(config.data_save_path):
        os.mkdir(config.data_save_path)
    print("Found path: {}".format(config.data_save_path))
    data_filename = str(config.data_filename.format(to_date_time(time.time())))
    print(data_filename)
    cwd = os.getcwd()
    print(cwd)
    full_file_path = os.path.join(cwd, config.data_save_path, data_filename)
    ## delete previous save.
    if os.path.isfile(full_file_path):
        'backing up old save_file'
        os.rename(full_file_path, '{}.old'.format(full_file_path))
    ## start the get and save download def.
    if not os.path.isfile(full_file_path):
        print("No dataset found, downloading new dataset. This will take some time.")
        print("New file being saved as {}".format(full_file_path))
        list_of_curpairs = []
        set_of_data = {}
        for i, e in enumerate(single_tick):
            if 'ZEC' in e:
                list_of_curpairs.append(e)
                historical_data = json_normalize(
                    quote_history(e,300,00,9999999999)
                )
                set_of_data['{}'.format(e)] = historical_data
            ## LET 'ER REST...
            time.sleep(.25)

    print(len(set_of_data))